In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing import image
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
def preprocess_image(img):
    """ Convert to grayscale, apply Gaussian blur to highlight features. """
    img = tf.image.rgb_to_grayscale(img)
    img = tf.image.adjust_contrast(img, 2)
    return img

# Setup data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_image,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    'D:/DATASET/CNN/ballooning/train',
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)


In [ ]:
# Initialize the ResNet50 base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Create a model for feature map extraction
feature_model = Model(inputs=base_model.input, outputs=base_model.get_layer('conv5_block3_out').output)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=7,
    validation_data=train_generator,
    validation_steps=len(train_generator)
)


In [ ]:
# Save the model
model.save('D:/DATASET/Models 7/model_ballooning7.h5')

In [ ]:
import json

# Save the training history
with open('D:/DATASET/Models 7/ballooning_history7.json', 'w') as file:
    json.dump(history.history, file)

In [ ]:
# Function to plot metrics
def plot_metrics(history, title):
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(train_acc) + 1)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title(f'Training and validation accuracy - {title}')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title(f'Training and validation loss - {title}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Plot metrics for each model
plot_metrics(history, 'Ballooning')

# Print final metrics
def print_final_metrics(history, title):
    train_acc = history.history['accuracy'][-1]
    val_acc = history.history['val_accuracy'][-1]
    train_loss = history.history['loss'][-1]
    val_loss = history.history['val_loss'][-1]

    print(f"{title} - Final Training Accuracy: {train_acc}")
    print(f"{title} - Final Validation Accuracy: {val_acc}")
    print(f"{title} - Final Training Loss: {train_loss}")
    print(f"{title} - Final Validation Loss: {val_loss}")

print_final_metrics(history, 'Ballooning')

In [ ]:
# Function to plot validation loss
def plot_validation_loss(history, model_name):
    plt.plot(history['val_loss'], label=f'{model_name} Validation Loss')
    plt.title(f'{model_name} Validation Loss Curve')
    plt.xlabel('Epochs')
    plt.ylabel('Validation Loss')
    plt.legend()
    plt.show()

In [ ]:
# Plot validation loss for ballooning model
with open('D:/DATASET/Models/ballooning_history7.json', 'r') as file:
    fibrosis_history = json.load(file)

plot_validation_loss(fibrosis_history, 'Ballooning')

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
plot_model(model, to_file='D:/DATASET/ballooning_model_architecture.png', show_shapes=True)

In [ ]:
for layer in model.layers:
    print(layer.name)

In [ ]:
# Create a model for feature map extraction
feature_model = Model(inputs=base_model.input, outputs=base_model.get_layer('conv2_block1_out').output)

In [ ]:
def extract_and_display_image(image_path, feature_model):
    img = Image.open(image_path)
    img_resized = img.resize((299, 299), Image.ANTIALIAS)
    img_array = np.array(img_resized) / 255.0
    img_array = img_array[np.newaxis, ...]

    # Display the original image
    plt.imshow(img)
    plt.title("Original Image")
    plt.axis('off')
    plt.show()

    # Get the feature maps
    feature_maps = feature_model.predict(img_array)[0]
    return feature_maps

In [ ]:
def visualize_feature_maps(feature_maps, num_features=15):
    # Display the first 'num_features' feature maps
    fig, axes = plt.subplots(nrows=int(np.sqrt(num_features)), ncols=int(np.sqrt(num_features)), figsize=(10, 10))
    for i, ax in enumerate(axes.flat):
        if i < num_features:
            ax.imshow(feature_maps[:, :, i], cmap='viridis')
            ax.axis('off')
    plt.show()

In [ ]:
def predict_and_display(image_path, model):
    img = image.load_img(image_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Create a batch
    img_array /= 255.0  # Normalize the image
    prediction = model.predict(img_array)

    # Display the processed image
    plt.imshow(img, cmap='gray')
    plt.title(f'Predicted: {"Ballooning Present" if prediction[0][0] > 0.5 else "Ballooning Absent"}')
    plt.show()

    return prediction

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image

image_path = "D:/DATASET/CNN/ballooning/train/1/14_206_22.png"
feature_maps = extract_and_display_image(image_path, feature_model)
visualize_feature_maps(feature_maps)
predicted_class=predict_and_display(image_path, model)
print("Probability of prediction= ",predicted_class)